In [1]:
import numpy as np
import scipy.sparse as sp
import time

In [2]:
import pandas as pd
user = pd.read_csv('D:\\social networks\\user.csv')
data_input = pd.read_csv('D:\\social networks\\gephi_input.csv')
f = open('D:\\social networks\\movie.csv')
movie = pd.read_csv(f) 

In [3]:
from sklearn.model_selection import train_test_split
user_train, user_test = train_test_split(user, test_size=0.2, random_state=0)

In [4]:
def socialMF(R,S,N,M,K,lambdaU,lambdaV,lambdaT,R_test,ul,il,utl,itl):#K算是隐特征个数#此处的S即为下面的T，是社交关系矩阵，目前只取了训练集上的
   #R是训练集上的打分矩阵，R_test是测试集上的打分矩阵，N是训练集users的个数，M是电影的个数，ul,il是训练集压缩矩阵表示非零元素位置的两个数
   #ul,il是测试集集压缩矩阵表示非零元素位置的两个数
    def sigmoid(z):
        return 1.0 / (1+np.exp(-z))
    def dsigmoid(z):
        return np.exp(-z)/np.power((1+np.exp(-z)),2)
    def rmse(U,V,R_test,utl,itl):
        error = (get_csrmat(sigmoid(U.dot(V.T)),utl,itl)-R).power(2).sum()/R.nnz
        return 5*np.sqrt(error)
    def mae(U,V,R_test,utl,itl):
        error = abs(get_csrmat(sigmoid(U.dot(V.T)),utl,itl)-R).sum()/R.nnz
        return error
    def get_csrmat(mat,ul,il):
        indx = ul*mat.shape[1]+il
        return sp.csr_matrix((np.take(np.array(mat),indx),(ul,il)),shape=(N,M))
    def costL(U,V):
        tmp = U.dot(V.T)
        Rx = get_csrmat(sigmoid(tmp),ul,il)
        cost = 0.5*((R - Rx).power(2)).sum()+0.5*lambdaU*np.linalg.norm(U)**2+0.5*lambdaV*np.linalg.norm(V)**2
        cost += 0.5*lambdaT*np.power(U-S.dot(U),2).sum()
        return cost
    def gradient(U,V):
        dU = np.zeros(U.shape)
        dV = np.zeros(V.shape)
        dU = lambdaU*U
        tmp = U.dot(V.T)
        Rv = get_csrmat(dsigmoid(tmp),ul,il)
        Rx = get_csrmat(sigmoid(tmp),ul,il)
        dU += Rv.multiply((Rx-R)).dot(V)
        dU += lambdaT*(U-S.dot(U))-lambdaT*S.T.dot((U-S.dot(U)))
        dV = lambdaV*V
        dV += (Rv.multiply((Rx-R))).T.dot(U)
        # print dU,dV
        if np.max(dU)>1:
            dU = dU/np.max(dU)
        if np.max(dV)>1:
            dV = dV/np.max(dV)
        return dU,dV

    def train(U,V):
        res=[]
        steps=5*10**3
        rate = 0.1
        pregradU = 0
        pregradV = 0
        tol=1e-3
        momentum = 0.9
        stage = max(steps/100 , 1)
        for step in range(steps):
            start = time.time()
            dU,dV = gradient(U,V)
            dU = dU + momentum*pregradU
            dV = dV + momentum*pregradV
            pregradU = dU
            pregradV = dV
            if not step%stage and rate>0.001:
                rate = 0.95*rate
            U -= rate * dU
            V -= rate * dV
            e = costL(U,V)
            res.append(e)
            if not step%stage:
                print (step,e,time.time() - start)
            if step>100 and abs(sum(res[-3:])-sum(res[-13:-10]))<tol:
                print("====================") 
                print("stop in %d step"%(step))
                print("error is ",e)
                print("====================")  
                break
        return U, V
    U = np.random.normal(0,0.01,size=(N,K))
    V = np.random.normal(0,0.01,size=(M,K))
    start = time.time()
    U,V = train(U,V)
    print ("=================RESULT=======================")
    print ('K:%d,lambdaU:%s, lambdaV:%s,lambdaT:%s' \
            %(K,lambdaU,lambdaV,lambdaT))
    print ("rmse",rmse(U,V,R_test,utl,itl))
    print ("mae",mae(U,V,R_test,utl,itl))
    print ("time",time.time() - start)
    print("U",U)
    print("V",V)
    return 0

In [5]:
N = 1000
idx = np.random.permutation(range(N))
user_train = user.loc[idx[:800]]
user_train = user_train.reset_index(drop=True)
user_test = user.loc[idx[800:]]
user_test = user_test.reset_index(drop=True)

In [6]:
N = len(user)
M = len(movie)
m = len(data_input)
a = user_train['name']
b = data_input.iloc[:,0]
c = data_input.iloc[:,1]
T= pd.DataFrame(columns = a, index = a) 
for i in range(m):
    if b[i] in set(user_train.name):
        if c[i] in set(user_train.name):
            T.loc[b[i],c[i]]=1

In [7]:
T = T.fillna(0)#此处只留了训练集的社会关系矩阵矩阵,现在还是DataFrame格式的
T =sp.dok_matrix(T)
T = T.tocsr()
print("get_T")
#================================================

get_T


In [9]:
import re
rates_dict = []
movie_id = []
for i in range(len(user_train)):
    str1 = user_train.rates[i]
    movie_id = re.findall(r"{u'(.+?)[': ]",str1)
    movie_id_s = re.findall(r", u'(.+?)[': ]",str1)
    movie_id.extend(movie_id_s)
    rate = re.findall(r": u'(.+?)'",str1)
    rate_float =  list(map(float,rate))#分数由字符型转为数值型
    a = []
    b = []
    for i,j in zip(movie_id,rate_float):
        if int(i) in set(movie.id):
            a.append(i)
            b.append(j)
    rates_d = dict(zip(a,b))
    rates_dict.append(rates_d)
r = pd.DataFrame(columns = movie.id, index = user_train.name)
for i in range(800):
    b = user_train.name
    c = list(rates_dict[i].keys())
    for j in range(len(c)):
        r.loc[b[i],int(c[j])]=rates_dict[i][c[j]]
r = r.fillna(0)
R =sp.dok_matrix(r)
R.tocsr()
print("get_R")

get_R


In [10]:
keylist = np.array(R.todok().keys()).T
ul = []
il = []
for i,j in list(keylist.tolist()):
    ul.append(i)
    il.append(j)
ul = np.array(ul)
il = np.array(il)
print("get_ul,il")

get_ul,il


In [11]:
rates_dict = []
movie_id = []
for i in range(len(user_test)):
    str1 = user_test.rates[i]
    movie_id = re.findall(r"{u'(.+?)[': ]",str1)
    movie_id_s = re.findall(r", u'(.+?)[': ]",str1)
    movie_id.extend(movie_id_s)
    rate = re.findall(r": u'(.+?)'",str1)
    rate_float =  list(map(float,rate))#分数由字符型转为数值型
    a = []
    b = []
    for i,j in zip(movie_id,rate_float):
        if int(i) in set(movie.id):
            a.append(i)
            b.append(j)
    rates_d = dict(zip(a,b))
    rates_dict.append(rates_d)
r = pd.DataFrame(columns = movie.id, index = user_test.name)
for i in range(200):
    b = user_test.name
    c = list(rates_dict[i].keys())
    for j in range(len(c)):
        r.loc[b[i],int(c[j])]=rates_dict[i][c[j]]
r = r.fillna(0)
R_test =sp.dok_matrix(r)
R_test.tocsr()
print("get_R_test")

get_R_test


In [12]:
keylist_R_test = np.array(R_test.todok().keys()).T
utl = []
itl = []
for i,j in list(keylist.tolist()):
    utl.append(i)
    itl.append(i)
utl = np.array(utl)
itl = np.array(itl)    
print("get_utl,itl")

get_utl,itl


In [18]:
N = 800
M = 1000
lambdaU,lambdaV,lambdaT,K = 0.01, 0.01, 0.02, 50
socialMF(R,T,N,M,K,lambdaU,lambdaV,lambdaT,R_test,ul,il,utl,itl)

0 152601.30054151555 0.1249699592590332
50 110321.57208575479 0.09373021125793457
100 110024.75087956396 0.10934782028198242
150 110005.5144943833 0.10934591293334961
200 110003.2958315714 0.09372615814208984
250 110002.54256636719 0.09372782707214355
300 110002.18536383769 0.09369921684265137
350 110001.98890608914 0.10934829711914062
400 110001.86993335397 0.10934948921203613
450 110001.79270605861 0.09372806549072266
500 110001.73988487545 0.10934877395629883
550 110001.7022415684 0.10934972763061523
600 110001.67450548724 0.10934734344482422
650 110001.6534959202 0.1249380111694336
700 110001.63720600627 0.10934877395629883
750 110001.6243216367 0.12497067451477051
800 110001.61395464657 0.1249992847442627
850 110001.60548805889 0.12496805191040039
900 110001.59848285126 0.12497115135192871
950 110001.59261991682 0.124969482421875
1000 110001.58766288002 0.12497091293334961
1050 110001.58343365832 0.10934782028198242
1100 110001.57979602837 0.14059162139892578
1150 110001.576644344

0